In [1]:
import torch
import pandas as pd

from datasets import Dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments, BitsAndBytesConfig

In [2]:
# QLoRA 설정을 위한 4bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bit 양자화 활성화
    bnb_4bit_use_double_quant=True,  # 이중 양자화 사용
    bnb_4bit_quant_type="nf4",  # 양자화 유형을 NF4로 설정
    bnb_4bit_compute_dtype=torch.bfloat16  # bfloat16을 사용해 계산
)

lora_config = LoraConfig(
    r=64,  # LoRA 랭크
    lora_alpha=16,  # LoRA 알파
    target_modules=["q_proj", "v_proj"],  # LoRA를 적용할 모듈
    lora_dropout=0.1,  # 드롭아웃 비율
    bias="none",  # 바이어스 사용 여부
    task_type="CAUSAL_LM"  # 작업 유형: 언어 모델
)

tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-bnb-4bit")

# 8B 크기의 사전 훈련된 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3-8b-bnb-4bit",  # 4bit 양자화된 모델 사용
    quantization_config=bnb_config,  # 양자화 설정 적용
    device_map="auto"  # 자동으로 GPU/CPU 할당
)

model = get_peft_model(model, lora_config)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [3]:
EOS_TOKEN = tokenizer.eos_token
alpaca_prompt = """
The following is a conversation between multiple people. Summarize the main points of the conversation.

### Conversation:
{}

### Summary:
{}"""


def formatting_prompts_func(examples):
    dialogues = examples["dialogue"]
    summaries = examples["summary"]
    texts = []
    for dialogue, summary in zip(dialogues, summaries):
        # dialogue와 summary를 Alpaca 포맷으로 변환하고 EOS_TOKEN을 추가합니다.
        text = alpaca_prompt.format(dialogue, summary) + EOS_TOKEN
        texts.append(text)

    # texts를 tokenizer로 인코딩하여 input_ids로 변환합니다.
    tokenized_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=128)

    return {
        "input_ids": tokenized_inputs['input_ids'],  # input_ids를 반환합니다.
        "attention_mask": tokenized_inputs['attention_mask'],  # attention_mask를 반환합니다.
        "labels": tokenized_inputs['input_ids'],  # input_ids를 labels로 사용 (언어 모델링의 경우)
    }

train_data = pd.read_csv("../dataset/cleaned_train.csv")
valid_data = pd.read_csv("../dataset/cleaned_dev.csv")

gen_df = pd.read_csv("../dataset/generated_train.csv")
gen_df = gen_df.dropna(subset=['dialogue', 'summary'])

trans_train = pd.read_csv("../dataset/processed_translated_train.csv")

train_data = pd.concat([train_data, gen_df, trans_train])

train_data = train_data[['dialogue', 'summary']]
valid_data = valid_data[['dialogue', 'summary']]

train_dataset = Dataset.from_pandas(train_data)
valid_dataset = Dataset.from_pandas(valid_data)

train_dataset = train_dataset.map(formatting_prompts_func, batched=True,)
valid_dataset = valid_dataset.map(formatting_prompts_func, batched=True,)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/44835 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [6]:
training_args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=8,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=-1,  # 최대 스텝 수
        do_eval=True,
        evaluation_strategy="epoch",  # 에폭 단위로 평가
        save_strategy="epoch",  # 에폭 단위로 모델 저장
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="./unsloth",  # 출력 디렉토리
        load_best_model_at_end=True  # 최적 모델을 마지막에 로드
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
)

trainer.train()

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import torch
from transformers import StoppingCriteria, StoppingCriteriaList, TextStreamer

# 모델과 토크나이저를 불러옵니다 (학습된 QLoRA 모델).
model = AutoModelForCausalLM.from_pretrained("./unsloth")  # 학습된 모델 디렉토리
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-bnb-4bit")

# EOS 토큰 설정
EOS_TOKEN = tokenizer.eos_token
alpaca_prompt = """
The following is a conversation between multiple people. Summarize the main points of the conversation.

### Conversation:
{}

### Summary:
{}"""

# 추론을 위한 정지 기준 클래스 정의
class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID 설정

    def __call__(self, input_ids, scores, **kwargs):
        return self.stop_token_id in input_ids[0]  # ID가 포함되면 중지

# end_token 설정
stop_token = "<|end_of_text|>"  
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[0]

# 정지 기준 리스트 설정
stopping_criteria = StoppingCriteriaList([StopOnToken(stop_token_id)])

# 추론 함수 정의
def generate_summary(dialogue):
    # 주어진 대화에 대해 요약을 생성합니다.
    prompt = alpaca_prompt.format(dialogue, "")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # 입력을 CUDA로 변환
    text_streamer = TextStreamer(tokenizer)  # 실시간 스트리밍을 위해 사용

    # 모델로부터 텍스트 생성
    generated_output = model.generate(
        **inputs,
        max_new_tokens=128,  # 최대 생성 토큰 수
        streamer=text_streamer,  # 실시간 출력
        stopping_criteria=stopping_criteria  # 정지 조건
    )

    # 생성된 텍스트를 디코딩하고 요약 부분 추출
    generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    summary = generated_text.split("### Summary:")[-1].strip()  # "### Summary:" 이후 부분만 추출
    return summary

# 예시 대화 데이터를 기반으로 요약 생성
test_data = pd.read_csv('../dataset/test.csv')  # 추론용 CSV 데이터 로드
result_data = []

for idx, row in test_data.iterrows():
    dialogue = row['dialogue']
    summary = generate_summary(dialogue)  # 요약 생성
    result_data.append({"fname": row['fname'], "summary": summary})

# 결과를 DataFrame으로 변환 후 CSV로 저장
result_df = pd.DataFrame(result_data)
result_df.to_csv("./unsloth/QLORA-predictions.csv", index=False)

# 불필요한 텍스트 부분을 제거하고 요약 부분만 추출
result_df['summary'] = result_df['summary'].apply(
    lambda x: x.split("### Summary:")[-1].split("<|end_of_text|>")[0].strip()
)

# 최종 결과 출력
print(result_df)
result_df.to_csv("./unsloth/QLORA-cleaned_predictions.csv", index=False)
